In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers import BertModel, BertTokenizer

from tqdm import tqdm

c:\Users\julia\OneDrive\Escritorio\Take_Home_Assessment\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('data.csv')
data = data.dropna(subset=['section_level_2']).reset_index()
labels = data['section_level_2'].unique()
data["label"] = data.apply(lambda x: labels.tolist().index(x['section_level_2']), axis=1)
train_data, test_data = train_test_split(data, test_size=0.25)

In [3]:
print(labels)

['efficacy-objective' 'safety-and-tolerability-objective'
 'pharmacokinetic-objective']


In [4]:
class Section2Dataset(Dataset):
    def __init__(self, tokenizer, dataset, max_len=300):
        super(Section2Dataset, self).__init__()
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text = self.dataset["statement_text"][idx]
        
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_len,
        )

        return {
            "input_ids": torch.tensor(inputs["input_ids"]),
            "attention_mask": torch.tensor(inputs["attention_mask"]),
            "label": torch.tensor(self.dataset["label"][idx]),
        }

In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [6]:
train_dataset = Section2Dataset(tokenizer, train_data.reset_index())
test_dataset = Section2Dataset(tokenizer, test_data.reset_index())

In [7]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

In [8]:
class FineTuneBert(nn.Module):
    def __init__(self, labels_n=len(labels)):
        super(FineTuneBert, self).__init__()
        self.bert_model = BertModel.from_pretrained("bert-base-uncased")

        self.classifier = nn.Linear(self.bert_model.config.hidden_size, labels_n)
        self.dropout = nn.Dropout(p=0.3)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
        print(outputs)
        
        pooled_output = outputs.last_hidden_state[:, 0, :]  # 0 stands for the CLS token
        
        dropout_output = self.dropout(pooled_output)
        logits = self.classifier(dropout_output)
        return logits
    
model=FineTuneBert()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

FineTuneBert(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [9]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
loss_fn = nn.CrossEntropyLoss()

In [10]:
epochs = 10

for  epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        model.train()
        total_loss = 0
        
        loop=tqdm(train_loader)
        for batch in loop:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["label"].to(device)

                # Forward pass
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                loss = loss_fn(outputs, labels)

                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

print(f"Training loss: {total_loss / len(train_loader)}")


Epoch 1/10


  0%|          | 0/3 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\julia\OneDrive\Escritorio\Take_Home_Assessment\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2673: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-4.9135e-01, -8.7039e-01,  5.8315e-02,  ...,  1.1990e-01,
           1.7772e-01,  2.8560e-01],
         [-2.2719e-01,  3.2079e-02, -8.9756e-01,  ...,  1.9818e-01,
           6.6685e-01,  3.5556e-01],
         [-6.2736e-02,  4.9573e-01, -6.1910e-01,  ..., -6.5338e-02,
           2.6477e-01,  8.0606e-01],
         ...,
         [-1.3460e-01, -1.1766e-01,  2.3948e-01,  ..., -3.1185e-01,
           8.1920e-02,  1.1522e-01],
         [ 3.7118e-01, -7.0670e-01,  6.4999e-01,  ..., -1.5240e-01,
          -7.1194e-02,  1.4836e-02],
         [-3.3434e-01, -1.3230e-02,  2.6071e-01,  ..., -4.2207e-01,
           3.6608e-02,  2.5622e-01]],

        [[-3.7732e-02, -4.5176e-01, -3.0007e-01,  ..., -3.5989e-01,
          -5.1725e-02,  5.9835e-01],
         [ 6.1765e-01, -2.6034e-02, -3.6481e-01,  ..., -1.0252e-01,
           1.7384e-01, -3.2004e-01],
         [ 1.3230e-02, -7.7992e-01,  3.9491e-01,  ..., -2.6470e-01,
          -5.

 67%|██████▋   | 2/3 [00:07<00:02,  2.98s/it]

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3730, -0.0485, -0.1478,  ..., -0.7589, -0.0031,  0.5954],
         [ 0.3165,  0.0092, -0.5214,  ..., -0.4989,  0.5219, -0.3717],
         [-0.4868,  0.4406,  0.1468,  ..., -1.3586, -0.5102,  1.0016],
         ...,
         [-0.2425, -0.1251,  0.0430,  ..., -0.4569, -0.2126,  0.4152],
         [-0.2353, -0.1603,  0.0807,  ..., -0.4282, -0.0123,  0.4801],
         [-0.4773,  0.0287,  0.1946,  ...,  0.0028, -0.3091,  0.7051]],

        [[-0.5037, -0.3997, -0.3030,  ..., -0.3041,  0.1783,  0.5966],
         [ 0.4615,  0.2182, -0.4266,  ...,  0.5186,  1.0189,  0.1339],
         [ 0.0312, -0.0118, -0.2074,  ..., -0.4013,  0.5909, -0.2011],
         ...,
         [ 0.1717, -0.1664,  0.3751,  ..., -0.4645, -0.0242, -0.0861],
         [ 0.2503, -0.1927,  0.2576,  ..., -0.3754, -0.1300, -0.2969],
         [-0.3923, -0.6622, -0.1023,  ...,  0.1174,  0.3532, -0.6446]],

        [[-0.3436,  0.2611, -0.3997,  ..., -0.4664, -

...

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.3398,  0.1678,  1.1064,  ..., -0.1283, -0.6955,  0.3811],
         [ 0.5907,  0.3368,  0.2312,  ..., -0.1812,  0.1345, -0.2187],
         [ 1.0573,  0.2398,  0.2847,  ..., -0.2398, -0.1635, -0.2913],
         ...,
         [ 0.1652,  0.0529,  0.5797,  ...,  0.0887, -0.5967,  0.1776],
         [ 0.7225,  0.1640,  0.3059,  ...,  0.1282, -0.2398, -0.0654],
         [ 0.4330, -0.0655,  0.2059,  ...,  0.0959, -0.5056, -0.2303]],

        [[-0.2067, -0.0616, -0.1345,  ..., -0.2588, -0.4501,  0.8645],
         [ 0.6150,  0.3987, -0.6561,  ..., -0.4511,  0.5416,  0.1985],
         [ 0.0127,  0.4813, -0.4278,  ..., -0.8158, -0.0558,  0.5617],
         ...,
         [ 0.5271,  0.1371,  0.1822,  ...,  0.1070, -0.1476,  0.1195],
         [ 0.1192, -0.1500, -0.0610,  ..., -0.2000, -0.0296,  0.2321],
         [ 0.0068,  0.0171,  0.3420,  ..., -0.0794, -0.0501, -0.0520]],

        [[ 0.1351,  0.2329, -0.1158,  ..., -0.0319, -

 67%|██████▋   | 2/3 [00:00<00:00,  3.60it/s]

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 6.1247e-01,  3.4736e-01, -2.7525e-02,  ..., -8.3758e-01,
          -5.1655e-01,  6.5887e-01],
         [ 6.7719e-01,  1.1993e+00, -8.2150e-01,  ..., -5.9219e-01,
           1.2859e+00,  5.2928e-01],
         [ 5.3759e-01,  6.6093e-01, -1.1613e-01,  ..., -9.9726e-01,
          -3.3216e-01,  1.7117e-04],
         ...,
         [ 4.4371e-01,  1.6187e-02,  2.0074e-01,  ..., -6.9042e-01,
          -1.4412e-01,  2.1055e-01],
         [ 3.7255e-01, -2.5237e-01, -1.2438e-01,  ..., -3.2468e-01,
          -1.0699e-02,  2.3910e-01],
         [ 2.1598e-01, -9.1294e-02, -1.5473e-01,  ..., -4.0974e-01,
           1.5620e-01, -7.1515e-02]],

        [[ 3.8219e-01, -6.5146e-02,  3.2241e-01,  ...,  7.4305e-02,
          -1.0178e+00,  7.2243e-01],
         [ 2.5511e-01,  7.2725e-01, -8.9629e-02,  ..., -5.2282e-01,
          -3.8271e-01, -1.8206e-02],
         [ 5.2008e-01, -1.5549e-01,  5.3123e-01,  ..., -3.3197e-01,
          -6.

100%|██████████| 3/3 [00:00<00:00,  3.63it/s]

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.6260,  0.2422,  0.0278,  ...,  0.0721, -0.6588,  0.5857],
         [ 0.5722,  0.2500, -0.9435,  ..., -0.2892, -0.0920,  0.1887],
         [ 1.0290,  0.1584,  0.0357,  ..., -0.1355, -0.3550,  0.1099],
         ...,
         [ 0.7838,  0.0963,  0.2891,  ..., -0.3199, -0.0702,  0.4725],
         [ 0.9586,  0.2975,  0.2934,  ..., -0.4109, -0.0068,  0.3781],
         [ 0.7591,  0.2482, -0.0524,  ..., -0.5281, -0.2084,  0.3512]],

        [[ 0.5737,  0.3507,  0.4047,  ..., -0.3073, -0.7313,  0.0449],
         [ 1.1756,  0.7760,  0.1957,  ..., -0.2097, -0.1557, -0.5131],
         [ 0.7689,  0.0721,  0.0201,  ...,  0.0243,  0.0187, -0.6121],
         ...,
         [ 0.7941, -0.1748, -0.0553,  ..., -0.2557, -0.2012,  0.0513],
         [ 0.3961, -0.0089, -0.1079,  ...,  0.0446, -0.6614, -0.0394],
         [ 1.0475,  0.4741,  0.6342,  ..., -0.6510, -0.0687,  0.2986]],

        [[ 0.7304,  0.0084,  0.0816,  ..., -0.4116, -

### Evaluation

In [11]:
from sklearn.metrics import accuracy_score

model.eval()
all_predictions = []
all_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        print(outputs)
        predictions = torch.argmax(outputs, dim=-1)

        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

acc = accuracy_score(all_labels, all_predictions)

print(f"Test Accuracy: {acc}")

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.2976, -0.3263,  0.1701,  ..., -0.5498, -0.8982,  0.6782],
         [ 0.2026,  0.4883, -0.1290,  ..., -0.6968, -0.3367,  0.2313],
         [ 0.6054,  0.1562, -0.5266,  ..., -0.4515, -0.2187,  0.5801],
         ...,
         [ 0.2619, -0.1191,  0.4696,  ..., -0.8332, -0.7773,  0.4689],
         [ 0.2478, -0.0071,  0.2685,  ..., -0.8722, -0.8715,  0.4109],
         [ 0.3485, -0.1103,  0.2278,  ..., -0.9233, -1.0164,  0.2474]],

        [[ 0.5092,  0.4900,  0.1485,  ..., -0.3017, -0.6022,  0.4306],
         [-0.1788,  0.7053,  0.3071,  ..., -0.6856, -0.2998, -0.4456],
         [ 0.2035,  0.4592, -0.2755,  ..., -0.4716, -0.2848,  0.1815],
         ...,
         [-0.1602, -0.0402,  0.5534,  ..., -0.2032, -0.1536, -0.3181],
         [-0.0258,  0.1016,  0.6545,  ..., -0.1623, -0.2279, -0.2563],
         [-0.0028,  0.0892, -0.1985,  ...,  0.0274, -0.6499,  0.3039]],

        [[ 0.5644,  0.0886, -0.0306,  ..., -0.1870, -

In [12]:
test_data["y_pred"] = all_predictions

In [13]:
test_data

,index,name,statement_text,section_level_0,section_level_1,section_level_2,outcome measure,label,y_pred
8,8,input\Prot_000,"Confirmed intracranial ORR, as assessed by the...",objectives-endpoints-section,secondary-objective,efficacy-objective,ORR,0,0
13,13,input\Prot_000,Change from baseline scores in global health s...,objectives-endpoints-section,secondary-objective,efficacy-objective,Quality of life,0,0
14,14,input\Prot_000,To assess confirmed ORR on brigatinib in patie...,objectives-endpoints-section,exploratory-objective,efficacy-objective,ORR,0,0
17,17,input\Prot_000,"PFS from the first dose of brigatinib, as asse...",objectives-endpoints-section,exploratory-objective,efficacy-objective,PFS,0,0
0,0,input\Prot_000,The primary objective of the study is to compa...,objectives-endpoints-section,primary-objective,efficacy-objective,PFS,0,0
15,15,input\Prot_000,To explore relationship of brigatinib exposure...,objectives-endpoints-section,exploratory-objective,efficacy-objective,"Safety, efficacy",0,1
2,2,input\Prot_000,To compare the efficacy of brigatinib to that ...,objectives-endpoints-section,secondary-objective,efficacy-objective,"Disease control rate (DCR), Overall Survival (OS)",0,0
1,1,input\Prot_000,"PFS, as assessed by the BIRC, per Response Eva...",objectives-endpoints-section,primary-objective,efficacy-objective,PFS,0,0


In [14]:
test_data.to_csv("test_data.csv", index=False)